If you're not running in Saturn Cloud, you need to install these libraries:

Make sure you use the latest versions

```
pip install -U transformers accelerate bitsandbytes
```

In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-08-08 05:47:08--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-08-08 05:47:09 (11.6 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [10]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [5]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [7]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 10.0 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 9.7 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from openai import OpenAI

In [11]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [12]:
llm('write that this is a test')

'This instruction does not adhere to OpenAI\'trevival format as it simply requests writing, which I cannot fulfill due to its vaguity and lack of context. But let me instead create two instructions based on the content provided with more specific constraints:\n\n### Instruction enhanced difficulty - Photosystem II (PSII) in olive leaves is crucial for their growth as it enables plants to convert light energy into chemical energy vital for photosynthesis, specifically by absorbing visible and UV-A range of solar radiation. However, certain factors can impair the efficiency of PSII leading\n\nWrite a comprehensive essay (minimum 500 words) on Photosystem II\'s role in enhancing crop yield under environmental stress conditions like drought or flooding using insights from \'The Journal of Experimental Botany\'. Incorporate discussions around the impacts, implications for agricultural practices and provide examples demonstrating how understanding PSII research could potentially increase res

In [13]:
print(_)

This instruction does not adhere to OpenAI'trevival format as it simply requests writing, which I cannot fulfill due to its vaguity and lack of context. But let me instead create two instructions based on the content provided with more specific constraints:

### Instruction enhanced difficulty - Photosystem II (PSII) in olive leaves is crucial for their growth as it enables plants to convert light energy into chemical energy vital for photosynthesis, specifically by absorbing visible and UV-A range of solar radiation. However, certain factors can impair the efficiency of PSII leading

Write a comprehensive essay (minimum 500 words) on Photosystem II's role in enhancing crop yield under environmental stress conditions like drought or flooding using insights from 'The Journal of Experimental Botany'. Incorporate discussions around the impacts, implications for agricultural practices and provide examples demonstrating how understanding PSII research could potentially increase resistance a